In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("../input/churn-modelling/Churn_Modelling.csv")

In [ ]:
df.head()

In [ ]:
sns.heatmap(df.corr(),annot=False)

**From the above figure we can say that Active members are not correlated with exit hence we can say that active members are not churned and age is slightly correlated with exit churn.**

# Independent and dependent variable**

In [ ]:
x=df.iloc[:,3:13]
y=df.iloc[:,13]

**We removed the rownumber, customer id and name as it doesnot give any impact to the determined dependent variable**

In [ ]:
x.head()

**Geography and Gender are of categorical type , we have convert into numerical variables for training**

In [ ]:
Gen=pd.get_dummies(x['Gender'],drop_first=True)

**We removed the first column of gender which will lead to reduce the dummy variable trap. As the male column is enought to describe the gender 0- female, 1- male**

In [ ]:
Gen.head()

**Likewise we have to do it for Geography variable**

In [ ]:
x['Geography'].unique()

In [ ]:
country=pd.get_dummies(x['Geography'],drop_first=True)

In [ ]:
country.head()

**We have to drop the geography and gender columns and add the dummy columns**

In [ ]:
x=x.drop(['Geography','Gender'],axis=1)

In [ ]:
x.head()

In [ ]:
x=pd.concat([x,Gen,country],axis=1)

In [ ]:
x.head()

**Now the data is ready for model training**

In [ ]:
#Hyperparameters

params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}
#These are the parameters of XGBoost

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost

In [ ]:
classifier=xgboost.XGBClassifier()

In [ ]:
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_jobs=-1,scoring='roc_auc',n_iter=5,cv=5,verbose=3)

In [ ]:
random_search.fit(x,y)

In [ ]:
random_search.best_estimator_

In [ ]:
random_search.best_params_

In [ ]:
classifier=xgboost.XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, gamma=0.1, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.2, max_delta_step=0, max_depth=3,
              min_child_weight=3, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

**Randomisedsearch CV gives the best hyperparameters for the xgboost**

In [ ]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(classifier,x,y,cv=10)

In [ ]:
score

In [ ]:
score.mean()